## Prepare example SHACL profiles from JSON API

In [1]:
import requests
import pprint
import pandas as pd

In [2]:
profile_id = 176 #198

In [3]:
profile_classes_url = 'https://ontome.net/api/classes-profile.json?lang=en&available-in-profile=' + str(profile_id)

try:
    response = requests.get(profile_classes_url)
    json_classes = response.json()
    print(len(str(json_classes)))
except Exception as e:
    print(e)


7187


In [4]:
### API output

# print(json_classes)

In [5]:
l = []

for cla in json_classes:
    l.append('@prefix ' + cla["namespacePrefix"] + ': <' + cla["namespaceURI"] + '>.')

classes_prefixes = list(set(l))    #list(pd.unique(pd.DataFrame(l)[0]))

pprint.pprint(classes_prefixes)

['@prefix crm: <http://www.cidoc-crm.org/cidoc-crm/>.',
 '@prefix sdh-so: <https://ontome.net/ns/social-legal-economic-life/>.']


In [6]:
### Classes and their elements

# classes list
lc = []
for cla in json_classes:
    lc.append([cla['namespacePrefix'] , cla["namespaceURI"], cla["classIdentifierInNamespace"], 
               cla['classID'], cla["classLabel"], cla["entityBasicType"], cla["ancestorClasses"]])

df_lc = pd.DataFrame(lc)
df_lc.columns = ['pref', 'ns_uri', 'uri', 'id', 'label', 'type', 'ancestors']
df_lc

,pref,ns_uri,uri,id,label,type,ancestors
0,crm,http://www.cidoc-crm.org/cidoc-crm/,E21,21,Person,8,"[1, 18, 19, 64, 66, 70, 83, 375, 380, 539, 881]"
1,crm,http://www.cidoc-crm.org/cidoc-crm/,E74,68,Group,8,"[1, 18, 27, 64, 65, 66, 70, 83, 375, 380, 539,..."
2,sdh-so,https://ontome.net/ns/social-legal-economic-life/,C5,442,Membership,9,"[1, 2, 211, 380, 699, 756]"
3,sdh-so,https://ontome.net/ns/social-legal-economic-life/,C2,444,Actor's Social Quality,30,"[1, 27, 53, 64, 65, 70, 380]"
4,sdh-so,https://ontome.net/ns/social-legal-economic-life/,C6,608,Membership Type,30,"[1, 27, 64, 65, 70, 380]"
5,sdh-so,https://ontome.net/ns/social-legal-economic-life/,C12,698,Actor's Social Role,30,"[1, 27, 53, 64, 65, 70, 380, 706]"


In [7]:

profile_properties_url = 'https://ontome.net/api/properties-profile.json?lang=en&available-in-profile=' + str(profile_id)

try:
    response = requests.get(profile_properties_url)
    json_properties = response.json()
    print(len(str(json_properties)))
except Exception as e:
    print(e)


7075


In [8]:
### API output

# print(json_properties)

In [9]:
l = []

for prop in json_properties:
    l.append('@prefix ' + prop["namespacePrefix"] + ': <' + prop["namespaceURI"] + '>.')


properties_prefixes = list(set(l)) #list(pd.unique(pd.DataFrame(l)[0]))

pprint.pprint(properties_prefixes)

['@prefix sdh-so: <https://ontome.net/ns/social-legal-economic-life/>.']


In [10]:
### properties and their elements

# properties list
lp = []
for prop in json_properties:
    lp.append([ prop["propertyDomain"], prop["domainInstancesMinQuantifier"], prop["domainInstancesMaxQuantifier"], 
               prop['namespacePrefix'], prop['namespaceURI'], 
               prop["propertyIdentifierInNamespace"], prop['propertyLabel'], 
               prop["propertyRange"], prop["rangeInstancesMinQuantifier"],prop["rangeInstancesMaxQuantifier"]])

df_lp = pd.DataFrame(lp)
df_lp.columns = ['id_s', 'min_s', 'max_s', 'ns_prefix', 'ns_uri', 'uri', 'label', 'id_t', 'min_t', 'max_t']
df_lp

,id_s,min_s,max_s,ns_prefix,ns_uri,uri,label,id_t,min_t,max_t
0,442,0,-1,sdh-so,https://ontome.net/ns/social-legal-economic-life/,P1,was a membership of,21,1,1
1,442,0,-1,sdh-so,https://ontome.net/ns/social-legal-economic-life/,P1,was a membership of,68,1,1
2,442,0,-1,sdh-so,https://ontome.net/ns/social-legal-economic-life/,P2,was membership in,68,1,1
3,442,0,-1,sdh-so,https://ontome.net/ns/social-legal-economic-life/,P3,has membership type,608,0,1
4,442,0,-1,sdh-so,https://ontome.net/ns/social-legal-economic-life/,P63,has quality during membership,444,0,-1
5,442,0,-1,sdh-so,https://ontome.net/ns/social-legal-economic-life/,P63,has quality during membership,698,0,-1


In [ ]:
## get domain classes
 
df_ms = df_lp.merge(df_lc[['pref', 'uri', 'id']], left_on='id_s', right_on='id', suffixes=['_ps', '_so'] )
df_ms

,id_s,min_s,max_s,ns_prefix,ns_uri,uri_ps,label,id_t,min_t,max_t,pref,uri_so,id
0,442,0,-1,sdh-so,https://ontome.net/ns/social-legal-economic-life/,P1,was a membership of,21,1,1,sdh-so,C5,442
1,442,0,-1,sdh-so,https://ontome.net/ns/social-legal-economic-life/,P1,was a membership of,68,1,1,sdh-so,C5,442
2,442,0,-1,sdh-so,https://ontome.net/ns/social-legal-economic-life/,P2,was membership in,68,1,1,sdh-so,C5,442
3,442,0,-1,sdh-so,https://ontome.net/ns/social-legal-economic-life/,P3,has membership type,608,0,1,sdh-so,C5,442
4,442,0,-1,sdh-so,https://ontome.net/ns/social-legal-economic-life/,P63,has quality during membership,444,0,-1,sdh-so,C5,442
5,442,0,-1,sdh-so,https://ontome.net/ns/social-legal-economic-life/,P63,has quality during membership,698,0,-1,sdh-so,C5,442


In [12]:
## get range classes

df_mt = df_ms.merge(df_lc[['pref', 'uri', 'id']], left_on='id_t', right_on='id', suffixes=['_pt', '_ta'] )
df_mt

,id_s,min_s,max_s,ns_prefix,ns_uri,uri_ps,label,id_t,min_t,max_t,pref_pt,uri_so,id_pt,pref_ta,uri,id_ta
0,442,0,-1,sdh-so,https://ontome.net/ns/social-legal-economic-life/,P1,was a membership of,21,1,1,sdh-so,C5,442,crm,E21,21
1,442,0,-1,sdh-so,https://ontome.net/ns/social-legal-economic-life/,P1,was a membership of,68,1,1,sdh-so,C5,442,crm,E74,68
2,442,0,-1,sdh-so,https://ontome.net/ns/social-legal-economic-life/,P2,was membership in,68,1,1,sdh-so,C5,442,crm,E74,68
3,442,0,-1,sdh-so,https://ontome.net/ns/social-legal-economic-life/,P3,has membership type,608,0,1,sdh-so,C5,442,sdh-so,C6,608
4,442,0,-1,sdh-so,https://ontome.net/ns/social-legal-economic-life/,P63,has quality during membership,444,0,-1,sdh-so,C5,442,sdh-so,C2,444
5,442,0,-1,sdh-so,https://ontome.net/ns/social-legal-economic-life/,P63,has quality during membership,698,0,-1,sdh-so,C5,442,sdh-so,C12,698


In [13]:
### new properties list

a = df_mt[['pref_pt', 'uri_so', 'min_s', 'max_s', 'ns_prefix', 'ns_uri', 'uri_ps', 'label', 'pref_ta', 'uri', 'min_t', 'max_t']].to_records()

prop_l = list(a)

pprint.pprint(prop_l)

[(0, 'sdh-so', 'C5', 0, -1, 'sdh-so', 'https://ontome.net/ns/social-legal-economic-life/', 'P1', 'was a membership of', 'crm', 'E21', 1, 1),
 (1, 'sdh-so', 'C5', 0, -1, 'sdh-so', 'https://ontome.net/ns/social-legal-economic-life/', 'P1', 'was a membership of', 'crm', 'E74', 1, 1),
 (2, 'sdh-so', 'C5', 0, -1, 'sdh-so', 'https://ontome.net/ns/social-legal-economic-life/', 'P2', 'was membership in', 'crm', 'E74', 1, 1),
 (3, 'sdh-so', 'C5', 0, -1, 'sdh-so', 'https://ontome.net/ns/social-legal-economic-life/', 'P3', 'has membership type', 'sdh-so', 'C6', 0, 1),
 (4, 'sdh-so', 'C5', 0, -1, 'sdh-so', 'https://ontome.net/ns/social-legal-economic-life/', 'P63', 'has quality during membership', 'sdh-so', 'C2', 0, -1),
 (5, 'sdh-so', 'C5', 0, -1, 'sdh-so', 'https://ontome.net/ns/social-legal-economic-life/', 'P63', 'has quality during membership', 'sdh-so', 'C12', 0, -1)]


### Prepare the output ttl file

In [14]:
### Namespaces list
ns_list = list(set(classes_prefixes + properties_prefixes))
ns_list_txt = '\n'.join(ns_list)
print(ns_list_txt)

@prefix crm: <http://www.cidoc-crm.org/cidoc-crm/>.
@prefix sdh-so: <https://ontome.net/ns/social-legal-economic-life/>.


In [15]:
print(lc[:1])

[['crm', 'http://www.cidoc-crm.org/cidoc-crm/', 'E21', 21, 'Person', 8, [1, 18, 19, 64, 66, 70, 83, 375, 380, 539, 881]]]


In [16]:
pprint.pprint(prop_l)

[(0, 'sdh-so', 'C5', 0, -1, 'sdh-so', 'https://ontome.net/ns/social-legal-economic-life/', 'P1', 'was a membership of', 'crm', 'E21', 1, 1),
 (1, 'sdh-so', 'C5', 0, -1, 'sdh-so', 'https://ontome.net/ns/social-legal-economic-life/', 'P1', 'was a membership of', 'crm', 'E74', 1, 1),
 (2, 'sdh-so', 'C5', 0, -1, 'sdh-so', 'https://ontome.net/ns/social-legal-economic-life/', 'P2', 'was membership in', 'crm', 'E74', 1, 1),
 (3, 'sdh-so', 'C5', 0, -1, 'sdh-so', 'https://ontome.net/ns/social-legal-economic-life/', 'P3', 'has membership type', 'sdh-so', 'C6', 0, 1),
 (4, 'sdh-so', 'C5', 0, -1, 'sdh-so', 'https://ontome.net/ns/social-legal-economic-life/', 'P63', 'has quality during membership', 'sdh-so', 'C2', 0, -1),
 (5, 'sdh-so', 'C5', 0, -1, 'sdh-so', 'https://ontome.net/ns/social-legal-economic-life/', 'P63', 'has quality during membership', 'sdh-so', 'C12', 0, -1)]


In [17]:
print(prop_l[2][4])

-1


In [20]:
### Write a file with the corresponding shacl profile

shacl_prof = """### SHACL OntoME Profile {} \n\n

@prefix sdh-shacl: <https://sdhss.org/shacl/profiles/>. 
@prefix sh: <http://www.w3.org/ns/shacl#> .  
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: 	<http://www.w3.org/2000/01/rdf-schema#> .  
""".format(profile_id) + ns_list_txt

## for each class
for c in lc: #[1:2]:
    
    if c[5] == 9:
        min_count_label_comment = ''
    else:
        min_count_label_comment = 'sh:minCount 1;'
    


    shacl_prof += """ \n
sdh-shacl:{0}_{1}_Shape a sh:NodeShape ;
    sh:targetClass {0}:{1} ;
    rdfs:label "{2}" ;

    sh:property [
        sh:path rdfs:label ;
        sh:datatype xsd:string;
        sh:order 1;
        {3}
        sh:maxCount 1;
        ];

    sh:property [
        sh:path rdfs:comment ;
        sh:datatype rdf:HTML;
        sh:order 2;
        {3}
        ];   

    """.format(c[0], c[2], c[4], min_count_label_comment)

    ## outgoing properties
    for p in prop_l:
        
        if p[12] == 1:
            max_count = 'sh:maxCount 1 ;'
        else:
            max_count = ''    

        if (p[1] == c[0] and p[2] == c[2]) :
            
            pl = """
        sh:property [
            sh:path {0}:{1} ;
            rdfs:label "{2}";
            sh:class {3}:{4} ;
            {5}
            ];
            """.format(p[5], p[7], p[8], p[9], p[10], max_count)


            shacl_prof += pl

    ## incoming properties
    for p in prop_l:

        
        if (p[9] == c[0] and p[10] == c[2]) :
            print(p[4])
            if p[4] == 1:
                max_count_i = 'sh:maxCount 1 ;'
            else:
                max_count_i = '' 

            pl = """
        sh:property [
            rdfs:label "Inverse of: {5}" ;
            {4}
            sh:path [ 
                sh:inversePath {0}:{1};
                sh:class {2}:{3};
                ] ;
            ];
            """.format(p[5], p[7], p[1], p[2], max_count_i, p[8])

            shacl_prof += pl

    shacl_prof += '.'


# print(shacl_prof)

## write file            
file_address = "shacl-profile-{}.ttl".format(profile_id)
with open(file_address, "w") as text_file:
    text_file.write(shacl_prof)


-1
-1
-1
-1
-1
-1


In [19]:
pprint.pprint(prop_l)

[(0, 'sdh-so', 'C5', 0, -1, 'sdh-so', 'https://ontome.net/ns/social-legal-economic-life/', 'P1', 'was a membership of', 'crm', 'E21', 1, 1),
 (1, 'sdh-so', 'C5', 0, -1, 'sdh-so', 'https://ontome.net/ns/social-legal-economic-life/', 'P1', 'was a membership of', 'crm', 'E74', 1, 1),
 (2, 'sdh-so', 'C5', 0, -1, 'sdh-so', 'https://ontome.net/ns/social-legal-economic-life/', 'P2', 'was membership in', 'crm', 'E74', 1, 1),
 (3, 'sdh-so', 'C5', 0, -1, 'sdh-so', 'https://ontome.net/ns/social-legal-economic-life/', 'P3', 'has membership type', 'sdh-so', 'C6', 0, 1),
 (4, 'sdh-so', 'C5', 0, -1, 'sdh-so', 'https://ontome.net/ns/social-legal-economic-life/', 'P63', 'has quality during membership', 'sdh-so', 'C2', 0, -1),
 (5, 'sdh-so', 'C5', 0, -1, 'sdh-so', 'https://ontome.net/ns/social-legal-economic-life/', 'P63', 'has quality during membership', 'sdh-so', 'C12', 0, -1)]
